# Homework Assignment #4 (Individual)
## Regression and SVM Predictive Models

### <p style="text-align: right;"> &#9989; Laura Pascoe
### <p style="text-align: right;"> &#9989; pascoela

### Goal for this homework assignment
By now, you have learned a bit about predictive models. In this assignment, you will practice:

* Building predictive models using `statsmodels` and `sklearn`
* Evaluating your predictive models

**This assignment is due roughly two weeks from now at 11:59 pm on Friday, April 21st.** It should be uploaded into the "Homework Assignments" submission folder for Homework #4.  Submission instructions can be found at the end of the notebook.


---
## Part 0: Add to your Git repository to track your progress on your assignment (4 points)

For this assignment, you're going to add it to the `cmse202-s23-turnin` repository you created in class so that you can track your progress on the assignment and preserve the final version that you turn in. In order to do this you need to

**&#9989; Do the following**:

1. Navigate to your `cmse202-s23-turnin` repository and create a new directory called `hw-04`.
2. Move this notebook into that **new directory** in your repository, then **add it and commit it to your repository**.
1. Finally, to test that everything is working, "git push" the file so that it ends up in your GitHub repository.

**Important**: Double check you've added your Professor and your TA as collaborators to your "turnin" repository (you should have done this in the previous homework assignment).

**Also important**: Make sure that the version of this notebook that you are working on is the same one that you just added to your repository! If you are working on a different copy of the notebook, **none of your changes will be tracked**!

If everything went as intended, the file should now show up on your GitHub account in the "`cmse202-s23-turnin`" repository inside the `hw-04` directory that you just created.  Periodically, **you'll be asked to commit your changes to the repository and push them to the remote GitHub location**. Of course, you can always commit your changes more often than that, if you wish.  It can be good to get into a habit of committing your changes any time you make a significant modification, or when you stop working on the project for a bit.

&#9989; **Do this**: Before you move on, put the command that your instructor should run to clone your repository in the markdown cell below.

git clone https://github.com/pascoela/CMSE202-s23-turnin

In [1]:
# imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC

# Building a Model to Predict if it Will Rain Tomorrow

In this dataset we have weather data for a number of Australian cities. We’re going to **build a model that uses data from today to predict if it will rain tomorrow.** To simplify our model, let’s just look at the weather for one city.

## Part 1. Working with the Data

The dataset that we’ll be using can be found here:
- `https://raw.githubusercontent.com/msu-cmse-courses/cmse202-S23-data/main/HW/HW4/aussie_weather_data.csv`

#### 1.1 (3 Points)
**Download the data set and read it in using Pandas.**

In [2]:
data = pd.read_csv("aussie_weather_data.csv")
data.head()

,Year,Month,DayOfMonth,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2009,1,1,NorfolkIsland,20.4,25.8,0.0,6.0,12.4,112.5,...,84.0,71.0,1014.5,1013.6,3.0,1.0,23.3,24.7,0,0
1,2009,1,2,NorfolkIsland,20.9,26.7,0.2,8.0,10.3,90.0,...,79.0,77.0,1016.3,1015.5,2.0,5.0,25.0,25.1,0,0
2,2009,1,3,NorfolkIsland,22.3,26.3,0.0,3.2,2.0,0.0,...,87.0,90.0,1014.6,1014.3,7.0,7.0,24.7,23.8,0,1
3,2009,1,4,NorfolkIsland,21.6,22.2,1.2,2.8,0.0,337.5,...,92.0,95.0,1016.0,1015.3,8.0,8.0,22.1,21.2,1,1
4,2009,1,5,NorfolkIsland,20.4,23.5,2.6,2.2,2.9,337.5,...,86.0,86.0,1015.3,1013.7,7.0,7.0,21.8,21.6,1,0


#### 1.2 (3 Points)
This dataset has weather information for multiple cities in Australia. It’s quite a large dataset. In fact, it’s a little *too* large for our purposes; it will take a considerable amount of time to train a model on so much data. So we’ll just work with the data for a specific city.

The first thing we’ll do is see which cities we have data for. 
**Print out a list of the specific cities in this dataset.** (Note that this is not the same as printing the entire `Location` column.)

In [3]:
cities = data['Location'].unique()
print(cities)

['NorfolkIsland' 'SydneyAirport' 'WaggaWagga' 'MelbourneAirport' 'Mildura'
 'Watsonia' 'Brisbane' 'Cairns' 'Townsville' 'MountGambier' 'Nuriootpa'
 'PerthAirport' 'Perth' 'AliceSprings' 'Darwin']


#### 1.3 (3 Points)
Select one of the cities (and **please** don't just pick the first one!). Cut down your dataframe so that it only contains data from your chosen city. You should also **look through the columns in this dataset and ensure that all of the data is ready for our analysis/model creation.**

In [4]:
perth = data[data['Location'] == 'Perth']
perth.head()

,Year,Month,DayOfMonth,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
31205,2008,7,2,Perth,6.4,20.7,0.0,1.8,7.0,67.5,...,80.0,39.0,1024.1,1019.0,0.0,6.0,11.1,19.7,0,0
31206,2008,7,4,Perth,9.5,19.2,1.8,1.2,4.7,180.0,...,93.0,73.0,1019.3,1018.4,6.0,6.0,13.2,17.7,1,1
31207,2008,7,5,Perth,9.5,16.4,1.8,1.4,4.9,202.5,...,69.0,57.0,1020.4,1022.1,7.0,5.0,15.9,16.0,1,1
31208,2008,7,6,Perth,0.7,15.9,6.8,2.4,9.3,45.0,...,86.0,41.0,1032.0,1029.6,0.0,1.0,6.9,15.5,1,0
31209,2008,7,7,Perth,0.7,18.3,0.0,0.8,9.3,90.0,...,72.0,36.0,1028.9,1024.2,1.0,5.0,8.7,17.9,0,0


#### 1.4 (3 Points)
Finally, before we start creating our models, let's first split our data into training and testing datasets. **Keep in mind what we want our model to predict and how this dataset gives you the information you need for your model.**

In [24]:
y = perth['RainTomorrow']
x = perth.drop(columns = ["RainTomorrow", "Location"])

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state = 530)

## 2.0 Building a Regression Model

#### 2.1 (3 Points)
We’ll start by creating a regression model. **Does it make sense for us to use linear regression or logistic regression? Explain your choice.**

I think it would make sense to use logisitic regression here since it predicts values between 0 and 1, and the RainTomorrow feature of the dataset is coded as 0 or 1.

#### 2.2 (12 Points)
Create a regression model. Use the model summary to determine which parameters are most statistically significant in your model. If you get an error during at this stage it may be helpful to go back to part 1.3 and ensure that all of your data is appropriate for creating a model. 

In [25]:
logit_model = sm.Logit(y_train, sm.add_constant(x_train)) 
result = logit_model.fit()
print(result.summary() )

Optimization terminated successfully.
         Current function value: 0.228007
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:           RainTomorrow   No. Observations:                 2268
Model:                          Logit   Df Residuals:                     2244
Method:                           MLE   Df Model:                           23
Date:                Thu, 20 Apr 2023   Pseudo R-squ.:                  0.5479
Time:                        11:18:38   Log-Likelihood:                -517.12
converged:                       True   LL-Null:                       -1143.7
Covariance Type:            nonrobust   LLR p-value:                1.469e-250
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const           115.3670     66.030      1.747      0.081     -14.049     244.783
Year              0.

The most statistically significant features are sunshine, windgustspeed, humidity3pm, pressure9am, and pressure3pm (p = 0.000 for all listed features)

#### 2.3 (8 Points)
**Fit your test data and create/print a confusion matrix. We’ll use this to evaluate how well your predicitve model performs.**

In [26]:
test_model = sm.Logit(y_test, sm.add_constant(x_test)) 
test_result = test_model.fit()

n_classes = 2
true_labels = y_test

pred_labels_not_rounded = test_result.predict()
pred_labels = pred_labels_not_rounded.round()

print(confusion_matrix(true_labels, pred_labels, labels = range(n_classes)))
print(classification_report(true_labels, pred_labels))

Optimization terminated successfully.
         Current function value: 0.237589
         Iterations 9
[[577  24]
 [ 50 106]]
              precision    recall  f1-score   support

           0       0.92      0.96      0.94       601
           1       0.82      0.68      0.74       156

    accuracy                           0.90       757
   macro avg       0.87      0.82      0.84       757
weighted avg       0.90      0.90      0.90       757



#### 2.4 (3 Points)
Look at/calculate the Precision and Recall of your model. **Which one is greater? Describe in plain language what that means about the performance of your model (I.e., the circumstances in which it does/doesn’t do well).**

I will examine the weighted avg of precision and recall since it accounts for any imbalances. Precision and recall are equal in that case (both .92), which means the model is equally as good at identifying true positives and avoiding false positives (e.g., the model is equally good at saying it will rain and it actually rains, and avoiding saying it will rain when it doesn't rain).

## 3.0 Interlude: A Base Rate Frequency Model (12 Points)

If we didn't create a fancy model for predicting future rainfall, would that mean that we had *no way* of predicting if it would rain? Of course not. There's always *some* kind of model we can use (they just won't be very sophisticated). 

In the case of our problem--predicting whether it will rain tomorrow--what is the simplest model we could use? We'd need to look at the **base rate frequency**. If we know that, over the course of the year, it rains ~30% of the time, that would be our base rate frequency. If we were to guess that it would be sunny every day of the year, we'd be wrong ~30% of the time, *but we'd be right ~70% of the time.* 

Why is this important? Imagine we create a sophisticated model that accurately predicts rain/no rain ~60% of the time; without any other information, we might think, "That's not too bad, I guess." It certainly feels better than *no* information, right? Now think about the base rate frequency model, which achieves an accuracy of **70%** just by guessing the same thing every time. Suddenly, our sophisticated model seems like crap! It tells us we could be doing ~10% better than our fancy model by guessing. This is why we use the base rate frequency as a simple way to test our models.


**Calculate how frequently it rains in your city. If you were to guess that it *wasn't* going to rain every day, how frequently would you be right? (We'll use this as our baseline model.)**


In [11]:
rain = perth[perth['RainToday'] == 1]
no_rain = perth[perth['RainToday'] == 0]

rain_probability = len(rain)/(len(rain)+len(no_rain))
no_rain_probability = len(no_rain)/(len(rain)+len(no_rain))

print("rain:" , rain_probability)
print("no rain:" , no_rain_probability)

rain: 0.19570247933884297
no rain: 0.804297520661157


If we guessed that it wasn't going to rain every day, we would be right about 80% of the time

---

### &#128721; STOP (3 Point)
**Pause to commit your changes to your Git repository!**

Take a moment to save your notebook, commit the changes to your Git repository with a meaningful commit message.

---

## 4.0 SVM Model

We’ve seen how regression faired in predicting rain; now let’s give Support Vector Machines a shot. 

It’s not clear, *a priori*, which type of kernel (linear or RBF) would fair better. Therefore, we’ll need to test out both kernel types, as well as multiple values for hyperparameters ($C$ and $\gamma$). 

#### 4.1 (12 Points)
Run a grid search over both types of kernels and multiple values of $C$ and $\gamma$ (we recommend [0.01,0.1,1.0,10.0] for both). We *STRONGLY* recommend you set the parameter `n_jobs=-1` in `GridSearchCV`, which will distribute the computational load. Make sure to write down your best-fit hyperparameters. 

In [27]:
tmp_vectors = x_train
tmp_labels = y_train

param_grid = {'C': [.01, .1, 1, 10],
              'gamma': [.01, .1, 1, 10],
              'kernel': ['linear','rbf']}

clf = GridSearchCV(SVC(class_weight='balanced'), param_grid, n_jobs = -1)

clf = clf.fit(tmp_vectors, tmp_labels)
print("Best estimator found by grid search:")
print(clf.best_estimator_)
print("Best parameters found by grid search:")
print(clf.best_params_)

Best estimator found by grid search:
SVC(C=0.01, class_weight='balanced', gamma=0.01, kernel='linear')
Best parameters found by grid search:
{'C': 0.01, 'gamma': 0.01, 'kernel': 'linear'}


the best fit hyper-parameters are C: .01, gamma: .01, and kernel: linear

#### 4.2 (8 Points)
Using your best-fit parameters, classify your test data and print off the confusion matrix. Note/calculate the Precision and Recall for your SVM model, as well as the overall accuracy.

$\mathrm{Accuracy} = \frac{\mathrm{Accurate~Predictions}}{\mathrm{Total~Samples}}$.

In [28]:
predict_vectors = x_test
true_labels = y_test
pred_labels = clf.predict(predict_vectors)

print(confusion_matrix(true_labels, pred_labels, labels=range(n_classes)))
print(classification_report(true_labels, pred_labels))

[[525  76]
 [ 24 132]]
              precision    recall  f1-score   support

           0       0.96      0.87      0.91       601
           1       0.63      0.85      0.73       156

    accuracy                           0.87       757
   macro avg       0.80      0.86      0.82       757
weighted avg       0.89      0.87      0.87       757



## 5. Evaluation of Models

Here we’re going to investigate how well our predictive models actually work.


#### 5.1 (5 Points)

Compare the accuracy of both of your predictive models (regression and SVM) to the baseline model from part 3. Do your predictive models do better than the baseline model? Is their performance relative to the baseline model what *you* would expect? 


Both predictive models do perform better than the baseline model (regression's accuracy was .92 and SVM's accuracy was .87 compared to the baseline of .80). Their performance is honestly about what I would expect; roughly 90% accuracy is generally pretty high, and since the baseline was already pretty high to begin with, I expected the predictive models to be better, but not by too much.

#### 5.2 (5 Points)

Let’s test the robustness of your model. Before you do anything, you should record the precision, recall, and accuracy of your two predictive models (if you haven’t already). Once you’ve done that, go all the way back to part 1.4 and change the random seed for `train_test_split`. Using this new data split, rerun all of your code from parts 2 and 4. For each new random seed, record the precision, recall, and accuracy for both models (and write them, as well as the random seed, in the cell below!). Do this for 3-5 different random seeds. 


Regression (Original)

    Precision: .92
    Recall: .92
    Accuracy: .92
    
SVM (Original) 

    Precision: .91
    Recall: .87
    Accuracy: .87
    
------------------------
    
Regression (Seed = 420)

    Precision: .94
    Recall: .94
    Accuracy: .94
    
SVM (Seed = 420) 

    Precision: .90
    Recall: .87
    Accuracy: .87
    
------------------------
    
Regression (Seed = 24)

    Precision: .91
    Recall: .92
    Accuracy: .92
    
SVM (Seed = 24) 

    Precision: .89
    Recall: .87
    Accuracy: .87
    
------------------------
    
Regression (Seed = 530)

    Precision: .90
    Recall: .90
    Accuracy: .90
    
SVM (Seed = 530) 

    Precision: .89
    Recall: .87
    Accuracy: .87

#### 5.3 (5 Points)

How much variation did you observe in the precision, recall, and accuracy in your models? After going through this exercise, how (if at all) does this change your response to part 5.1? 

There was little variation across different random seeds. This does not change my response to part 5.1; even with the differing seeds, the models always still performed better than the baseline.

#### 5.4 (5 Points)

Finally, compare the performance of your regression and SVM models. Which, if any, performed better? Esxplain.

Both models performed relatively similarly, although the regression model consistently had slightly higher precision, recall, and accuracy, even across all tested random seeds

---

### &#128721; STOP (3 Point)
**Pause to commit your changes to your Git repository!**

Take a moment to save your notebook, commit the changes to your Git repository with a meaningful commit message.

---

### Congratulations, you're done!

&#169; Copyright 2022,  Department of Computational Mathematics, Science and Engineering at Michigan State University